In [1]:
from keras.models import Sequential
from keras.layers import Dense,InputLayer
from tensorflow.keras.optimizers import Adam
from keras.losses import MeanSquaredError
from keras.metrics import RootMeanSquaredError
from keras_tuner import RandomSearch


/home/david/anaconda3/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/david/anaconda3/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/david/anaconda3/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/david/anaconda3/lib/python3.7/site-packages/tensorboard/co

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
df=pd.read_csv("df_new",index_col=[0])
df

,carat,cut,color,depth,table,volume,price
0,-1.199402,-0.971068,-1.548072,-0.173495,-1.100486,-1.172273,326
1,-1.241651,-0.181133,-1.548072,-1.362393,1.585691,-1.219563,326
2,-1.199402,0.608803,-1.548072,-3.390512,3.376475,-1.173874,327
3,-1.072656,-0.181133,-1.060350,0.455922,0.242603,-1.063233,334
4,-1.030407,0.608803,-0.572629,1.085338,0.242603,-0.996797,335
...,...,...,...,...,...,...,...
53789,-0.164306,-0.971068,1.378257,-0.663041,-0.205093,-0.177929,2757
53790,-0.164306,0.608803,1.378257,0.945468,-1.100486,-0.149908,2757
53791,-0.206555,1.398738,1.378257,0.735662,1.137995,-0.196740,2757
53792,0.131436,-0.181133,-0.084907,-0.523171,0.242603,0.139958,2757


In [3]:
X=df.drop("price",1)
Y=df[["price"]]
X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.2, random_state=42)


In [11]:
def find_model(hp):
    model=Sequential()
    model.add(InputLayer(6))
    for i in range(hp.Int("num_layers",2,10)):
        model.add(Dense(units=hp.Int("units" + str(i),min_value=32,max_value=512,step=32),
                        activation="relu"))
    model.add(Dense(1,activation="linear"))
    model.compile(optimizer=Adam(hp.Choice("learning rate",[1e-2,1e-2])),loss=MeanSquaredError(),
                  metrics= MeanSquaredError())
    return model              
                  

In [12]:
tuner=RandomSearch(
    find_model,
    objective="val_mean_squared_error",
    max_trials=5,
    executions_per_trial=1,
    project_name="neural_network",
    overwrite=True
      
)

In [13]:
tuner.search(X_train,y_train, epochs=4,validation_data=(X_test,y_test))

Trial 5 Complete [00h 00m 23s]
val_mean_squared_error: 1481954.125

Best val_mean_squared_error So Far: 1442437.5
Total elapsed time: 00h 02m 40s
INFO:tensorflow:Oracle triggered exit


In [7]:
# tuner.search_space_summary()# to get the search of the tuner


In [10]:
tuner.results_summary()#to get summary of the result

Results summary
Results in ./neural_network
Showing 10 best trials
Objective(name="val_mean_squared_error", direction="min")

Trial 3 summary
Hyperparameters:
num_layers: 3
units0: 256
units1: 480
learning rate: 0.01
units2: 160
Score: 1466460.75

Trial 4 summary
Hyperparameters:
num_layers: 6
units0: 352
units1: 96
learning rate: 0.01
units2: 224
units3: 32
units4: 32
units5: 32
Score: 1532815.25

Trial 0 summary
Hyperparameters:
num_layers: 3
units0: 224
units1: 256
learning rate: 0.01
units2: 32
Score: 1550328.125

Trial 1 summary
Hyperparameters:
num_layers: 2
units0: 160
units1: 64
learning rate: 0.01
units2: 288
Score: 1617279.0

Trial 2 summary
Hyperparameters:
num_layers: 2
units0: 448
units1: 288
learning rate: 0.01
units2: 64
Score: 1631909.75


In [9]:
# tuner.get_best_models()[0]